In [ ]:
%load_ext autoreload
%autoreload 2

In [107]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import seaborn as sns

import os

from imblearn.over_sampling import SMOTE
from collections import Counter

from pcap_info import *

%matplotlib inline

In [2]:
sns.set()

In [108]:
rfpath = './reduced_features'
red_feat_file = './reduced_feature_list.txt'

In [109]:
with open(red_feat_file) as f:
    red_feat = list(map(lambda l: l.strip(), f.readlines()))
print(red_feat)

['ratetot_num', 'ratetot_sz', 'ratet_num_0', 'ratet_num_2', 'ratet_sz_0', 'ratet_sz_2', 'rate_num_0', 'rate_num_2', 'rate_num_4', 'rate_num_6', 'rate_num_16', 'rate_num_18', 'rate_num_20', 'rate_num_34', 'rate_num_40', 'rate_num_44', 'rate_num_48', 'rate_num_52', 'rate_num_54', 'rate_sz_0', 'rate_sz_2', 'rate_sz_4', 'rate_sz_6', 'rate_sz_16', 'rate_sz_18', 'rate_sz_20', 'rate_sz_34', 'rate_sz_40', 'rate_sz_44', 'rate_sz_48', 'rate_sz_52', 'rate_sz_54', 'fract_num_0', 'fract_num_2', 'fract_sz_0', 'fract_sz_2', 'frac_num_0', 'frac_num_2', 'frac_num_4', 'frac_num_16', 'frac_num_18', 'frac_num_20', 'frac_num_34', 'frac_num_40', 'frac_num_44', 'frac_num_48', 'frac_num_52', 'frac_sz_0', 'frac_sz_2', 'frac_sz_4', 'frac_sz_16', 'frac_sz_18', 'frac_sz_20', 'frac_sz_34', 'frac_sz_40', 'frac_sz_44', 'frac_sz_48', 'frac_sz_52', 'rrattot_num', 'rrattot_sz', 'rratt_num_0', 'rratt_num_2', 'rratt_sz_0', 'rratt_sz_2', 'rrat_num_2', 'rrat_num_18', 'rrat_num_34', 'rrat_num_44', 'rrat_num_48', 'rrat_num_5

In [110]:
def shuffle_unison(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

In [111]:
all_data = {}
for i, dev in enumerate(all_device_list):
    dev_data = []
    for pcap in pcaps[dev]:
        fn = os.path.join(rfpath, os.path.basename(pcap) + '.txt')
        with open(fn) as f:
            arr = map(lambda l: list(map(float, l.strip().split()[1:])),
                      f.readlines())
            dev_data.extend(arr)
    classes = [i] * len(dev_data)
    all_data[dev] = (dev_data, classes)

In [112]:
train_data = {}
test_data = {}
for dev, (feat, cls) in all_data.items():
    np.random.shuffle(feat)
    train_amt = int(np.floor(len(feat) * 0.8))
    train_data[dev] = (feat[:train_amt][:], cls[:train_amt])
    test_data[dev] = (feat[train_amt:][:], cls[train_amt:])

In [113]:
print(train_data['wemo'][0])

[[19.28, 7171.44, 0.0, 19.28, 0.0, 7171.44, 0.0, 12.36, 0.0, 0.0, 0.0, 0.0, 0.0, 6.92, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6064.28, 0.0, 0.0, 0.0, 0.0, 0.0, 1107.16, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.641078838174, 0.0, 0.0, 0.0, 0.0, 0.358921161826, 0.0, 0.0, 0.0, 0.0, 0.0, 0.845615385473, 0.0, 0.0, 0.0, 0.0, 0.154384614527, 0.0, 0.0, 0.0, 0.0, 0.404564315353, 0.164892964314, 0.0, 0.595435684647, 0.0, 0.835107035686, 0.711974110032, 0.0, 0.387283236994, 0.0, 0.0, 0.0, 0.909219231302, 0.0, 0.429170128979, 0.0, 0.0, 0.0, 0.595435684647, 0.835107035686, 0.0, 0.404564315353, 0.0, 0.164892964314, 0.288025889968, 0.0, 0.0, 0.0, 0.612716763006, 0.0, 0.0, 0.090780768698, 0.0, 0.0, 0.0, 0.570829871021, 0.0, 0.0, 371.962655602, 0.0, 371.962655602, 0.0, 490.637540453, 0.0, 0.0, 0.0, 0.0, 0.0, 159.994219653, 0.0, 0.0, 0.0, 0.0, 0.0, 486.440017365, -0.0, 486.440017365, -0.0, 567.925049371, -0.0, -0.0, -0.0, -0.0, 116.869361429, -0.0, -0.0, -0.0, -0.0], [5.8, 4010.52, 0.0, 5.8, 0.0, 4010

In [116]:
rf_clf = GridSearchCV(estimator=RandomForestClassifier(),
                      param_grid=dict(n_estimators=list(range(8, 22, 2)),
                                      max_depth=[None, 60, 50, 40, 30, 20]), n_jobs=-1)
tr = ([], [])
ts = ([], [])
for dev in all_device_list:
    tr[0].extend(train_data[dev][0])
    tr[1].extend(train_data[dev][1])
    ts[0].extend(test_data[dev][0])
    ts[1].extend(test_data[dev][1])
scaler = StandardScaler().fit(tr[0])
rf_clf.fit(scaler.transform(tr[0]), tr[1])

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [8, 10, 12, 14, 16, 18, 20], 'max_depth': [None, 60, 50, 40, 30, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [118]:
rf_clf.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [117]:
confusion = np.zeros((len(all_device_list), len(all_device_list)))
for tsv, tsc in zip(scaler.transform(ts[0]), ts[1]):
    prd = rf_clf.predict(np.array(tsv).reshape(1, -1))[0]
    confusion[prd][tsc] += 1
print('All trained:')
print(confusion.astype(int))

All trained:
[[355   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0 364   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0 345   0   0   0   7   1   0  27   0   0   0   0]
 [  0   0   0 354   0   0   0   0   0   0   0   1   0   0]
 [  0   0   0   0 352   0   0   0   0   0   0   0   2   0]
 [  0   0   0   0   0 357   0   0   0   0   0   0   0   0]
 [  0   0   1   0   0   0 339  13   0   5   0   0   0   0]
 [  0   0   1   0   0   0   8 339   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0 356   0   0   0   0   0]
 [  0   0  11   0   0   0   2   3   0 322   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0 375   0   0   0]
 [  0   0   0   1   5   0   0   0   0   0   0 355   0   0]
 [  0   0   0   1   0   0   0   0   0   0   0   0 348   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0 384]]


In [54]:
# Train on nine plugs, test on the remainder
grouped_train_data_exclude = {}
grouped_test_data_exclude = {}
for out in device_list:
    collected_train = ([], [])
    collected_test = ([], [])
    for dev in all_device_list:
        if dev not in [out] + non_switch + non_actuated:
            collected_train[0].extend(train_data[dev][0])
            collected_train[1].extend([0] * len(train_data[dev][1]))
        elif dev != out:
            collected_train[0].extend(train_data[dev][0])
            collected_train[1].extend(train_data[dev][1])
        
        if dev == out:
            collected_test[0].extend(test_data[dev][0])
            collected_test[1].extend([0] * len(test_data[dev][1]))
        #elif dev in non_switch + non_actuated:
            #collected_test[0].extend(test_data[dev][0])
            #collected_test[1].extend(test_data[dev][1])

    shuffle_unison(*collected_train)
    shuffle_unison(*collected_test)
    
    sm = SMOTE(random_state=42)
    collected_train = sm.fit_sample(*collected_train)

    grouped_train_data_exclude[out] = collected_train
    grouped_test_data_exclude[out] = collected_test

In [125]:
# Train on nine plugs, test on the remainder
grouped_train_data_exclude = {}
grouped_test_data_exclude = {}
for out in device_list:
    collected_train = ([], [])
    collected_test = ([], [])
    for dev in all_device_list:
        if dev not in [out] + non_switch + non_actuated:
            collected_train[0].extend(train_data[dev][0])
            collected_train[1].extend([0] * len(train_data[dev][1]))
        elif dev != out:
            collected_train[0].extend(train_data[dev][0])
            collected_train[1].extend(train_data[dev][1])
        
        if dev == out:
            collected_test[0].extend(test_data[dev][0])
            collected_test[1].extend([0] * len(test_data[dev][1]))
        #elif dev in non_switch + non_actuated:
            #collected_test[0].extend(test_data[dev][0])
            #collected_test[1].extend(test_data[dev][1])

    shuffle_unison(*collected_train)
    shuffle_unison(*collected_test)
    
    sm = SMOTE(random_state=42)
    collected_train = sm.fit_sample(*collected_train)

    grouped_train_data_exclude[out] = collected_train
    grouped_test_data_exclude[out] = collected_test

In [126]:
clfs = []
for out in device_list:
    clf = GridSearchCV(estimator=RandomForestClassifier(),
                       param_grid=dict(n_estimators=list(range(8, 22, 2)),
                                       max_depth=[None, 60, 50, 40, 30, 20]), n_jobs=-1)
    tr = grouped_train_data_exclude[out]
    ts = grouped_test_data_exclude[out]
    clf.fit(tr[0], tr[1])
    print('Excluding {}:'.format(out), clf.score(ts[0], ts[1]))
    clfs.append(clf)

Excluding wemo: 0.101408450704
Excluding neo: 1.0
Excluding jinvoo: 1.0
Excluding ihome: 0.325842696629
Excluding dlink: 0.358543417367
Excluding tplink: 0.0
Excluding mjerry: 1.0
Excluding tuya: 1.0
Excluding edimax: 0.0421348314607
Excluding cevitor: 1.0


In [127]:
def explain_subtype(f):
    parts = f.split('_')
    if len(parts) <= 2: return f
    st = int(parts[-1])
    if parts[0] in ['ratet', 'fract', 'sratt', 'lt', 'sdt']:
        if st == 0:
            return '{} ({})'.format(f, 'Management')
        elif st == 1:
            return '{} ({})'.format(f, 'Control')
        elif st == 2:
            return '{} ({})'.format(f, 'Data')
    else:
        return '{} ({})'.format(f, frame_types[st])

In [128]:
forest = rf_clf.best_estimator_
importances = list(zip(red_feat, forest.feature_importances_))
list(map(lambda t: (explain_subtype(t[0]), t[1]), sorted(importances, key=lambda t: t[1], reverse=True)))

[('ratet_num_2 (Data)', 0.047194020702481757),
 ('sdt_sz_2 (Data)', 0.04502667909097758),
 ('srat_num_2 (Data - Data)', 0.041707806372950962),
 ('rate_num_34 (Data - QoS Data)', 0.041147949281786653),
 ('ratetot_num', 0.039947919705241627),
 ('sd_sz_34 (Data - QoS Data)', 0.03942690341855766),
 ('srat_num_34 (Data - QoS Data)', 0.036497502944243664),
 ('rate_sz_34 (Data - QoS Data)', 0.036320135337886424),
 ('l_sz_34 (Data - QoS Data)', 0.033521881434639897),
 ('ltot_sz', 0.032859455081756153),
 ('rate_num_2 (Data - Data)', 0.030470034291507901),
 ('sd_sz_2 (Data - Data)', 0.027832169174088595),
 ('ratet_sz_2 (Data)', 0.02764832623771064),
 ('ratetot_sz', 0.025849003279744549),
 ('sdtot', 0.023296235764790969),
 ('rate_num_18 (Data - Null)', 0.023225039560582368),
 ('srat_sz_2 (Data - Data)', 0.022240410172753512),
 ('lt_sz_2 (Data)', 0.02154068333342812),
 ('srat_sz_34 (Data - QoS Data)', 0.021277235311616273),
 ('frac_sz_34 (Data - QoS Data)', 0.02002589112318174),
 ('frac_num_2 (Dat

In [129]:
for out, clf in zip(device_list, clfs):
    ts = grouped_test_data_exclude[out]
    confusion = np.zeros((len(all_device_list), len(all_device_list)))
    for tsv, tsc in zip(ts[0], ts[1]):
        prd = clf.predict(np.array(tsv).reshape(1, -1))[0]
        confusion[prd][tsc] += 1
    print('Excluding {}:'.format(out))
    print(confusion.astype(int))

Excluding wemo:
[[ 36   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [319   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
Excluding neo:
[[364   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   

### SVC, Linear Kernel

In [119]:
svc_clf = GridSearchCV(estimator=SVC(),
                       param_grid=dict(kernel=['linear'], C=[0.6, 0.8, 1.0, 1.2, 1.4, 1.6],
                                       random_state=[42]), n_jobs=-1)
tr = ([], [])
ts = ([], [])
for dev in all_device_list:
    tr[0].extend(train_data[dev][0])
    tr[1].extend(train_data[dev][1])
    ts[0].extend(test_data[dev][0])
    ts[1].extend(test_data[dev][1])
svc_clf.fit(scaler.transform(tr[0]), tr[1])

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': [0.6, 0.8, 1.0, 1.2, 1.4, 1.6], 'random_state': [42], 'kernel': ['linear']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [120]:
confusion = np.zeros((len(all_device_list), len(all_device_list)))
for tsv, tsc in zip(scaler.transform(ts[0]), ts[1]):
    prd = svc_clf.predict(np.array(tsv).reshape(1, -1))[0]
    confusion[prd][tsc] += 1
print('All trained:')
print(confusion.astype(int))

All trained:
[[355   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0 364   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0 232   0   0   0  14   5   0 111   0   0   0   0]
 [  0   0   0 330   0   0   0   0   0   0   0  17   0  13]
 [  0   0   0   1 287   0   0   0   6   0   0  70   2   1]
 [  0   0   0   0   0 357   0   0   0   0   0   2   0   0]
 [  0   0  50   0   0   0 265  56   0   8   0   0   0   0]
 [  0   0  19   0   0   0  62 286   0   7   0   0   0   0]
 [  0   0   0   0   3   0   0   0 350   0   0   3   0   0]
 [  0   0  57   0   0   0  15   9   0 228   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0 375   0   0   0]
 [  0   0   0  24  67   0   0   0   0   0   0 264   0   5]
 [  0   0   0   1   0   0   0   0   0   0   0   0 348   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0 365]]


### SVC, RBF Kernel

In [130]:
clf = GridSearchCV(estimator=SVC(),
                   param_grid=dict(kernel=['rbf'], gamma=['auto', 0.05, 0.1],
                                   random_state=[42]), n_jobs=-1)
tr = ([], [])
ts = ([], [])
for dev in all_device_list:
    tr[0].extend(train_data[dev][0])
    tr[1].extend(train_data[dev][1])
    ts[0].extend(test_data[dev][0])
    ts[1].extend(test_data[dev][1])
clf.fit(tr[0], tr[1])

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'gamma': ['auto', 0.05, 0.1], 'random_state': [42], 'kernel': ['rbf']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [131]:
confusion = np.zeros((len(all_device_list), len(all_device_list)))
for tsv, tsc in zip(ts[0], ts[1]):
    prd = clf.predict(np.array(tsv).reshape(1, -1))[0]
    confusion[prd][tsc] += 1
print('All trained:')
print(confusion.astype(int))

All trained:
[[247   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0 334   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0 311   0   0   0   1   2   0  21   0   0   0   0]
 [  0   0   0 329   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0 312   0   0   0   0   0   0   1   0   0]
 [  0   0   0   0   0 348   0   0   0   0   0   0   0   0]
 [  0   0   4   0   0   0 311  11   0   2   0   0   0   0]
 [  0   0   0   0   0   0  14 325   0   1   0   0   0   0]
 [  0   0   0   0   0   0   0   0 303   0   0   0   0   0]
 [  0   0  11   0   0   0   2   5   0 269   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0 163   0   0   0]
 [  0   0   0   2   0   0   0   0   0   0   0 336   0   0]
 [  0   0   0   1   0   0   0   0   0   0   0   0 297   0]
 [108  30  32  24  45   9  28  13  53  61 212  19  53 384]]


### MLP Classifier

In [122]:
mlp_params = [dict(hidden_layer_sizes=[(90,)], random_state=[42]),
              dict(hidden_layer_sizes=[(70,)], random_state=[42]),
              dict(hidden_layer_sizes=[(50,)], random_state=[42]),
              dict(hidden_layer_sizes=[(30,)], random_state=[42]),
              dict(hidden_layer_sizes=[(90,70)], random_state=[42]),
              dict(hidden_layer_sizes=[(90,50)], random_state=[42]),
              dict(hidden_layer_sizes=[(90,30)], random_state=[42]),
              dict(hidden_layer_sizes=[(70,50)], random_state=[42]),
              dict(hidden_layer_sizes=[(90,70,50)], random_state=[42])]

mlp_clf = GridSearchCV(estimator=MLPClassifier(),
                       param_grid=mlp_params, n_jobs=-1)
tr = ([], [])
ts = ([], [])
for dev in all_device_list:
    tr[0].extend(train_data[dev][0])
    tr[1].extend(train_data[dev][1])
    ts[0].extend(test_data[dev][0])
    ts[1].extend(test_data[dev][1])
mlp_clf.fit(scaler.transform(tr[0]), tr[1])

GridSearchCV(cv=None, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'random_state': [42], 'hidden_layer_sizes': [(90,)]}, {'random_state': [42], 'hidden_layer_sizes': [(70,)]}, {'random_state': [42], 'hidden_layer_sizes': [(50,)]}, {'random_state': [42], 'hidden_layer_sizes': [(30,)]}, {'random_state': [42], 'hidden_layer_sizes': [(90, 70)]}, {'random_s...2], 'hidden_layer_sizes': [(70, 50)]}, {'random_state': [42], 'hidden_layer_sizes': [(90, 70, 50)]}],
       pre_dispatch='2*n_jobs', refit=True, retu

In [123]:
confusion = np.zeros((len(all_device_list), len(all_device_list)))
for tsv, tsc in zip(scaler.transform(ts[0]), ts[1]):
    prd = mlp_clf.predict(np.array(tsv).reshape(1, -1))[0]
    confusion[prd][tsc] += 1
print('All trained:')
print(confusion.astype(int))

All trained:
[[353   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0 364   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0 253   0   0   0  20   8   0  50   0   0   0   0]
 [  1   0   0 342   0   0   0   0   0   0   0   3   0   5]
 [  0   0   0   1 350   0   0   0  10   0   0  26   4   1]
 [  0   0   0   1   1 357   0   0   0   0   0   1   0   0]
 [  0   0  24   0   0   0 282  51   0  17   0   0   0   0]
 [  1   0  29   0   0   0  44 296   0   4   0   0   0   0]
 [  0   0   0   0   0   0   0   0 346   0   0   0   0   0]
 [  0   0  52   0   0   0  10   1   0 283   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0 375   0   0   0]
 [  0   0   0  11   6   0   0   0   0   0   0 326   0   8]
 [  0   0   0   1   0   0   0   0   0   0   0   0 346   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0 370]]


In [124]:
mlp_clf.best_estimator_

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(70, 50), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=42, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)